In [10]:
import numpy as np
import random as r
import matplotlib.pyplot as plt
import scipy.signal as s
from math import sin, cos, exp, sqrt, cosh, log, pi
import math 
import scipy.fft as fft
import cmath
from ipywidgets import *
import plotly.graph_objects as go

import statsmodels.api as sm
lowess = sm.nonparametric.lowess

from scipy.signal import butter, filtfilt
################################################################

SEED = r.randrange(1, 100)

checkbox = widgets.Checkbox(
    value=False,
    description='set seed',
    disabled=False,
    indent=False,
    layout=Layout(width='100px',margin='0 0 0 30px')
)

slider = widgets.IntSlider(
    value=0,
    min=0,
    max=1,
    step=1,
)


seed_slider = widgets.IntSlider(
    value=1,
    min=1,
    max=10,
    step=1,
    description='seed:',
    layout=Layout(width='300px')
)


slider_T =  widgets.IntSlider(
    value=0,
    min=0,
    max=10,
    step=1,
    description='param:'
)

slider_N =  widgets.IntSlider(
    value=100,
    min=100,
    max=100000,
    step=10,
    description='n:'
)



fig1 = go.Figure( layout=go.Layout(
       font=dict(family="Courier New, monospace",size=18,color="#7f7f7f"),width=600, height=600,margin=dict(l=0, r=0, t=0, b=0) ,
    legend=dict(x=0, y=1)))


In [11]:

def BFG1(n):
    
    X = np.transpose([*np.zeros(n)])
    
    N = 0.7
    F_max = 0.1
    sigma = math.sqrt(0.3 * N * F_max)
    
    
    S = np.random.standard_normal((2,n))
    X_new = X + S * sigma
    XX=[]
    for i in range(0,X.size):
        XX.append(i)
    
    return [[XX,X_new[0]],[XX,X]]
    
    
def BFG2(n):
    cutoff_freq = 0.2
    b,a = butter(6,cutoff_freq, btype='low', analog=False)#'lowpass')    
    Filt = filtfilt(b, a, BFG1(n)[0][1])
    XX=[]
    for i in range(0,Filt.size):
        XX.append(i)
    
    return [[XX,Filt]]
   

def BFG3_4(n):
    # ачх баттерворта 6-го порядка
    w = np.transpose([*np.zeros(600)])
    m = 0
    for i in np.arange(0.0, 6.0, 0.01):
        w[m] = i
        m = m + 1
    Hf = np.transpose([*np.zeros(600)])
    
    for i in range(600):
        Hf[i] = 1/(1+((i/100)/1)**(2*6))
    
    fx = np.fft.fft(BFG2(n)[0][1])
  
    fx_2 = abs(fx*fx)
    fx_3 = np.hstack((fx_2[:int(len(fx_2)/2)]))
    fx_4 = fx_3/n
    
    
    
    XX = []
    for i in range(0, len(fx_4)):
        XX.append(i)
        
    #print(XX)
    
    return [[w,Hf],[XX,fx_4]]

def BFG3(n):
    return [BFG3_4(n)[0]]

def BFG4(n):
    return [BFG3_4(n)[1]]






def css_widget(text, font_size=10):
    html = "<h1 style='font-size:"+str(font_size)+"px; margin-left: 50px; margin-top: 0;margin-bottom:0;width:350px'>" + str(text) + "</h1>"
    W = widgets.HTML(value=html)
    return W


In [12]:

step_btn = widgets.Button(description='refresh', button_style='primary')
reset_btn = widgets.Button(description='reset', button_style='primary')

def custom_layout(w,h):

    ll=Layout(width=str(w) + 'px', height=str(h) + 'px', border='solid')

    return ll 


def showText(text):
    print("\t", text)


def function(widget, figure, fun_for_data):
    
    global SEED
    
    figure.data = []   
    
    if(checkbox.value == True):
        np.random.seed(seed_slider.value)
    else:
        np.random.seed(SEED)
 
    data = fun_for_data() 
    
    for values in data:
        figure.add_trace(go.Scatter(x=values[0], y=values[1]))
        


    figure.show()
    
    
    

def function(widget, figure, fun_for_data, lisii_and_tiny):
    
    global SEED
    
    lisii_and_tiny = slider_N.value
    
    figure.data = []   
    
    if(checkbox.value == True):
        np.random.seed(seed_slider.value)
    else:
        np.random.seed(SEED)
 
    data = fun_for_data(lisii_and_tiny) 
    
    for values in data:
        figure.add_trace(go.Scatter(x=values[0], y=values[1]))
        
        

    figure.show()


    
        
    


def custom_display():
    
    text=""" На цифровой сигнал подается БГШ (белый гаусс. шум). Пропускаем через фильтр Баттерворта. Рассчитать отношения мощностей до и после фильтрации.
    В работе используются два фильтра: фильтр Баттерворта и встроенный в пакет scipy.signal lowess """
    
    
    HH_ui1 = VBox([interactive_plot1])
    HH_ui2 = VBox([interactive_plot2])
    HH_ui3 = VBox([interactive_plot3])
    HH_ui4 = VBox([interactive_plot4])
    
    
    tab = widgets.Tab(children = [ HH_ui1, HH_ui2,HH_ui3,HH_ui4])
    tab.set_title(0, '1')
    tab.set_title(1, '2')
    tab.set_title(2, '3')
    tab.set_title(3, '4')
    tab.set_title(4, '5')
    tab.set_title(5, '6')
    
    
    ui = VBox([slider_T,slider_N, 
               HBox([checkbox, seed_slider], layout =custom_layout("auto", "auto") ), 
               HBox([step_btn,reset_btn], layout = Layout(  display='flex',
                                                            flex_flow='row',
                                                            justify_content='space-between'))
              ])
    
    
    
    tab2 = widgets.Tab(children = [css_widget("Задача 5",15), ui])
    tab2.set_title(0, 'Теория')
    tab2.set_title(1, 'Параметры')

    
    

    
    HHH_2 = (Box([tab], layout = Layout(display='flex',
                    flex_flow='column',
                    align_items='stretch',
                    border='solid',
                    width='60%')))
    
    
    
    #     Htext = VBox([css_widget("Задача 1",15),
    #                   css_widget(text,12),
    #                   css_widget(text_for_slider1,12),

    #                   out3,
    #                   css_widget(text_for_slider2,12),
    #                   out4,

    #                   css_widget(text_for_slider3,12),
    #                   out5])

    
    HHH_3 = (Box([tab2], layout=Layout(display='flex',
                    flex_flow='column',
                    align_items='stretch',
                    border='solid',
                    width='40%')))
    
    item_2 = HBox([HHH_2,HHH_3])

    display(VBox([item_2]))
    
    toggle_visibility(None)


def toggle_visibility(button):
    seed_slider.layout.visibility = 'hidden' if (checkbox.value == False) else  'visible'


def clicker(b):
    global SEED
    SEED = r.randrange(1, 100)
    slider.value = 1 if (slider.value == 0) else  0  
    custom_display()
       
step_btn.on_click(clicker)
checkbox.observe(toggle_visibility)


БОЛТ_and_tiny = 100


interactive_plot1 = interactive_output(function, {'widget':slider, 'figure':fixed(fig1), 'fun_for_data': fixed(BFG1),
                                                  'lisii_and_tiny':fixed(БОЛТ_and_tiny)})
interactive_plot2 = interactive_output(function, {'widget':slider, 'figure':fixed(fig1), 'fun_for_data': fixed(BFG2),
                                                  'lisii_and_tiny':fixed(БОЛТ_and_tiny)})
interactive_plot3 = interactive_output(function, {'widget':slider, 'figure':fixed(fig1), 'fun_for_data': fixed(BFG3),
                                                  'lisii_and_tiny':fixed(БОЛТ_and_tiny)})
interactive_plot4 = interactive_output(function, {'widget':slider, 'figure':fixed(fig1), 'fun_for_data': fixed(BFG4),
                                                  'lisii_and_tiny':fixed(БОЛТ_and_tiny)})
#interactive_plot3 = interactive_output(function, {'widget':slider, 'figure':fixed(fig1), 'fun_for_data': fixed(BFG3) })
#interactive_plot4 = interactive_output(function, {'widget':slider, 'figure':fixed(fig1), 'fun_for_data': fixed(BFG4) })
#interactive_plot5 = interactive_output(function, {'widget':slider, 'figure':fixed(fig1), 'fun_for_data': fixed(BFG5) })
#interactive_plot6 = interactive_output(function, {'widget':slider, 'figure':fixed(fig1), 'fun_for_data': fixed(BFG6) })

custom_display()

In [ ]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:50% !important; }</style>"))